In [14]:
import numpy as np
import torch
import dxtb
from dxtb.typing import DD
from dxtb.config import ConfigCache
from dxtb import OutputHandler
from tblite.interface import Calculator

dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}

# LiH
numbers = torch.tensor([3, 1], device=dd["device"])
positions = torch.tensor([[0.0, 0.0, 0.0], [0.0, 0.0, 1.5]], **dd) # ** to use dd as kwargs 

numbers = torch.tensor([6, 6, 7, 7, 1, 1, 1, 1, 1, 1, 8, 8,], device=dd["device"])
positions = torch.tensor([
                [-3.81469488143921, +0.09993441402912, 0.00000000000000],
                [+3.81469488143921, -0.09993441402912, 0.00000000000000],
                [-2.66030049324036, -2.15898251533508, 0.00000000000000],
                [+2.66030049324036, +2.15898251533508, 0.00000000000000],
                [-0.73178529739380, -2.28237795829773, 0.00000000000000],
                [-5.89039325714111, -0.02589114569128, 0.00000000000000],
                [-3.71254944801331, -3.73605775833130, 0.00000000000000],
                [+3.71254944801331, +3.73605775833130, 0.00000000000000],
                [+0.73178529739380, +2.28237795829773, 0.00000000000000],
                [+5.89039325714111, +0.02589114569128, 0.00000000000000],
                [-2.74426102638245, +2.16115570068359, 0.00000000000000],
                [+2.74426102638245, -2.16115570068359, 0.00000000000000],
                ], **dd) # ** to use dd as kwargs

pos = positions.clone().requires_grad_(True)

# instantiate a dxtb calculator
cache_config = ConfigCache(enabled=True, coefficients=True, mo_energies=True, density=True)
# cache_config = ConfigCache(enabled=False, coefficients=False, mo_energies=False, density=False) # does not fix auto jac
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, CACHE_STORE_DENSITY=True, **dd)
calc.opts.cache = cache_config
OutputHandler.verbosity = 5

# Trimmed down singlepoint function

In [15]:
from dxtb import labels
from dxtb._src import scf
from dxtb.config import Config
from dxtb.integrals import DriverManager
from dxtb._src.integral.container import IntegralMatrices
from dxtb import IndexHelper
from dxtb._src.components.interactions.coulomb.secondorder import new_es2
from dxtb._src.components.interactions.coulomb.thirdorder import new_es3
from dxtb._src.components.interactions.list import InteractionList
from dxtb._src.xtb.gfn1 import GFN1Hamiltonian
from dxtb._src.constants import defaults

from dxtb.integrals import DriverManager
from dxtb.integrals.factories import new_overlap
from tad_mctc.convert import any_to_tensor

def trimmed_singlepoint(numbers, positions, result="density", chrg=defaults.CHRG, spin=defaults.SPIN):
    # Device and dtype
    device = positions.device
    dtype = positions.dtype
    dd = {'device': device, 'dtype': dtype}

    chrg = any_to_tensor(chrg, **dd)
    if spin is not None:
        spin = any_to_tensor(spin, **dd)

    opts = Config(**dd)
    opts.ints.level = labels.INTLEVEL_HCORE
    opts.exclude = set()  # Make sure 'scf' is not excluded

    par = dxtb.GFN1_XTB
    ihelp = IndexHelper.from_numbers(numbers, par)


    # Integral matrices
    intmats = IntegralMatrices(**dd)

    ## Overlap integral
    driver_name = 0 # libcint
    drv_mgr = DriverManager(driver_name, **dd)
    drv_mgr.create_driver(numbers, par, ihelp)
    drv_mgr.driver.setup(positions)

    ovlp_integral = new_overlap(drv_mgr.driver_type, **dd)
    ovlp_integral.build(drv_mgr.driver)

    intmats.overlap = ovlp_integral.matrix # Same return as overlap(pos)

    ## Hcore integral
    h0 = GFN1Hamiltonian(numbers, par, ihelp, **dd)
    hcore = h0.build(positions, ovlp_integral.matrix)
    intmats.hcore = hcore


    # Create interactions (es2 and es3)
    es2 = (
        new_es2(numbers, par, **dd)
        if not {"all", "es2"} & opts.exclude
        else None
    )
    es3 = (
        new_es3(numbers, par, **dd)
        if not {"all", "es3"} & opts.exclude
        else None
    )
    interactions = InteractionList(es2, es3, **dd)

    # Build interaction cache
    icaches = interactions.get_cache(numbers=numbers, positions=positions, ihelp=ihelp)

    scf_opts = opts.scf

    # Run SCF
    scf_results = scf.solve(
        numbers,
        positions,
        chrg,
        spin,
        interactions,
        icaches,
        ihelp,
        scf_opts,
        intmats,
        h0.refocc,
    )

    return scf_results["density"]

# Run the calculation
scf_results = trimmed_singlepoint(numbers, pos, chrg=0, spin=None)

print(scf_results)


iter  Energy                   Delta E         Delta Pnorm     Delta q        
-----------------------------------------------------------------------------
  0   -2.33649843882747E+01    0.000000E+00    0.000000E+00    0.000000E+00
  1   -2.34634144776997E+01    9.843009E-02    6.602257E+00    7.644933E-01
  2   -2.34980108875033E+01    3.459641E-02    2.079152E+00    2.903451E-01
  3   -2.34980608116288E+01    4.992413E-05    1.079661E-01    1.092139E-02
  4   -2.34980648424925E+01    4.030864E-06    2.843219E-02    3.237257E-03
  5   -2.34980650497666E+01    2.072741E-07    7.919470E-03    7.440992E-04
  6   -2.34980650663530E+01    1.658647E-08    2.734669E-03    1.997555E-04
  7   -2.34980650668001E+01    4.470522E-10    4.005738E-04    2.838210E-05
-----------------------------------------------------------------------------



tensor([[ 7.6460e+00, -5.3877e-01, -1.1403e-01,  ..., -5.2305e-02,
          8.8480e-03, -1.5675e-16],
        [-5.3877e-01,  2.4950e+00, -2.9682e-02,  ...,  6.8161e-03,
          2.8034e-03, -9.8943e-17],
        [-1.1403e-01, -2.9682e-02,  1.9471e+00,  ...,  4.3013e-02,
         -4.3929e-03, -5.3157e-16],
        ...,
        [-5.2305e-02,  6.8161e-03,  4.3013e-02,  ...,  7.2002e+00,
         -1.2475e+00,  8.1278e-17],
        [ 8.8480e-03,  2.8034e-03, -4.3929e-03,  ..., -1.2475e+00,
          5.4789e+00, -2.1209e-16],
        [-1.5675e-16, -9.8943e-17, -5.3157e-16,  ...,  8.1278e-17,
         -2.1209e-16,  5.9599e+00]], dtype=torch.float64,
       grad_fn=<ReshapeAliasBackward0>)


In [19]:
def get_density(pos):
    return trimmed_singlepoint(numbers, pos, result="density")

density_jac = torch.autograd.functional.jacobian(get_density, pos)

print(density_jac)


iter  Energy                   Delta E         Delta Pnorm     Delta q        
-----------------------------------------------------------------------------
  0   -2.33649843882747E+01    0.000000E+00    0.000000E+00    0.000000E+00
  1   -2.34634144776997E+01    9.843009E-02    6.602257E+00    7.644933E-01
  2   -2.34980108875033E+01    3.459641E-02    2.079152E+00    2.903451E-01
  3   -2.34980608116288E+01    4.992413E-05    1.079661E-01    1.092139E-02
  4   -2.34980648424925E+01    4.030864E-06    2.843219E-02    3.237257E-03
  5   -2.34980650497666E+01    2.072741E-07    7.919470E-03    7.440992E-04
  6   -2.34980650663530E+01    1.658647E-08    2.734669E-03    1.997555E-04
  7   -2.34980650668001E+01    4.470522E-10    4.005738E-04    2.838210E-05
-----------------------------------------------------------------------------



tensor([[[[-6.9774e-01,  2.1143e-01, -4.1210e-16],
          [-6.6223e-03,  5.1248e-03, -9.3948e-19],
          [ 1.4524e+00, -2.0869e+00,  3.6340e-17],
          ...,
          [ 3.6673e-03, -1.7498e-04, -6.4579e-18],
          [ 1.0601e+00,  1.5998e+00,  3.7812e-16],
          [ 8.1100e-03, -7.4243e-04, -8.2607e-18]],

         [[-2.1057e-01, -2.4749e-01, -8.6371e-16],
          [ 1.4307e-02,  9.6439e-03, -1.6931e-17],
          [-1.6222e-01, -5.1021e-01,  2.7638e-16],
          ...,
          [-7.8492e-03, -5.4614e-04,  3.0659e-17],
          [-2.3012e-01,  5.4742e-01,  4.1524e-16],
          [-1.8378e-02, -9.6968e-03, -2.1407e-17]],

         [[-1.9095e-01, -1.6997e+00, -7.8842e-16],
          [-5.1939e-03, -7.1694e-02,  1.2250e-17],
          [-8.2120e-01,  1.1615e+00,  1.3284e-16],
          ...,
          [ 4.2115e-04,  4.6227e-03, -4.6015e-18],
          [ 8.1106e-01,  1.0365e+00,  3.5882e-16],
          [ 3.8426e-02,  3.9952e-02,  4.2893e-18]],

         ...,

         [[ 6.88

# See what is needed with verbosity

In [5]:
calc.get_density(pos)

Singlepoint 
 - Classicals        ... done
 - Overlap           ... done
 - Core Hamiltonian  ... done
 - Interaction Cache ... done

Starting SCF Iterations...

iter  Energy                   Delta E         Delta Pnorm     Delta q        
-----------------------------------------------------------------------------
  0   -8.77560617962661E-01    0.000000E+00    0.000000E+00    0.000000E+00
  1   -8.95077734935744E-01    1.751712E-02    2.772240E-01    2.663464E-01
  2   -8.96548332460670E-01    1.470598E-03    9.872227E-02    9.003866E-02
  3   -8.96556230914213E-01    7.898454E-06    5.901386E-03    5.292786E-03
  4   -8.96556871688813E-01    6.407746E-07    1.956400E-03    1.756005E-03
  5   -8.96556883308107E-01    1.161929E-08    2.858123E-04    2.564900E-04
  6   -8.96556883341053E-01    3.294598E-11    1.527107E-05    1.370057E-05
-----------------------------------------------------------------------------

SCF finished in 6 iterations.
SCF Energy  : -0.89655688330811 Hartree.

tensor([[ 2.8233e-01,  4.1713e-17, -4.1713e-17,  1.3667e-01,  5.0646e-01,
          7.6675e-03],
        [ 4.1713e-17,  6.1630e-33, -6.1630e-33,  2.0193e-17,  7.4827e-17,
          1.1328e-18],
        [-4.1713e-17, -6.1630e-33,  6.1630e-33, -2.0193e-17, -7.4827e-17,
         -1.1328e-18],
        [ 1.3667e-01,  2.0193e-17, -2.0193e-17,  6.6160e-02,  2.4517e-01,
          3.7117e-03],
        [ 5.0646e-01,  7.4827e-17, -7.4827e-17,  2.4517e-01,  9.0851e-01,
          1.3754e-02],
        [ 7.6675e-03,  1.1328e-18, -1.1328e-18,  3.7117e-03,  1.3754e-02,
          2.0823e-04]], dtype=torch.float64, grad_fn=<ReshapeAliasBackward0>)